# MR課題
> 一部、加工したファイル(`レポート用アンケート調査 copy.xlsx`)を利用する

In [1]:
from sklearn import linear_model
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## データの読み込みと可視化

In [2]:
input_path  = "レポート用アンケート調査 copy.xlsx"
df = pd.read_excel(input_path, sheet_name="データシート")

In [5]:
df.head()

,No,性別,シャープ_信頼できる,シャープ_親しみやすい,シャープ_個性的である,シャープ_洗練されてる,シャープ_都会的である,シャープ_先進性がある,シャープ_性能が優れている,シャープ_デザインが良い,...,生活全般について_友人たちと何かをするときは，誘うことより誘われることの方が多い,生活全般について_人に助言するのは得意なほうではないと思う,生活全般について_自分は熱しやすく冷めやすいタイプだと思う,生活全般について_多くの人が訪れる話題のスポットなどへ出かけるのが好きだ,生活全般について_刺激的だがリスクがある生活よりも，リスクの少ない安定した生活を送りたいと思う,生活全般について_他人と争ってでも自己主張するより，自己主張せずに周囲とのトラブルを避けたい,生活全般について_余暇時間を大事にし，充実させたいと強く思う,生活全般について_仕事が生きがいで，仕事中心に生活したいと思う,生活全般について_自分が周囲からどのように思われるかが気になる,生活全般について_周囲から一目置かれたいと思う
0,1,True,1,2,2,2,1,1,1,2,...,2,1,1,2,2,2,2,1,2,2
1,2,True,4,4,4,4,4,4,4,4,...,5,5,4,4,2,2,5,5,5,4
2,3,True,4,4,2,3,3,2,3,2,...,4,3,5,2,3,2,4,4,4,4
3,4,True,5,5,4,5,3,5,5,5,...,4,2,4,1,4,4,5,2,4,5
4,5,True,3,2,1,3,2,2,3,3,...,1,3,2,2,2,3,1,3,2,3


In [4]:
# 「性別」列をbool型に変換（男性:True, 女性:False と仮定）
df["性別"] = df["性別"].map({"男性": True, "女性": False})


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 91 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   No                                               52 non-null     int64  
 1   性別                                               52 non-null     bool   
 2   シャープ_信頼できる                                       52 non-null     int64  
 3   シャープ_親しみやすい                                      52 non-null     int64  
 4   シャープ_個性的である                                      52 non-null     int64  
 5   シャープ_洗練されてる                                      52 non-null     int64  
 6   シャープ_都会的である                                      52 non-null     int64  
 7   シャープ_先進性がある                                      52 non-null     int64  
 8   シャープ_性能が優れている                                    52 non-null     int64  
 9   シャープ_デザインが良い                      

In [49]:
comp_names = ["シャープ", "ソニー ","パナソニック","日立", "東芝", "三菱電機"]
for comp_name in comp_names:
    comp_cols = [col for col in df.columns if col.startswith(f"{comp_name}_")]
    comp_data = df[comp_cols]
    #display(comp_data.head())
    comp_means = comp_data.mean()
    #print(comp_means)
    plt.figure(figsize=(10, 6))
    comp_means.plot(kind='bar')
    plt.title(f"ブランド『{comp_name}』に対する平均イメージ評価")
    plt.ylim(2.5,4.5)
    plt.ylabel("平均スコア")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig(f"output/comp_eval_means_bar_graph/comp_means_{comp_name}.png")
    plt.close()

In [60]:
df.head()

,No,性別,シャープ_信頼できる,シャープ_親しみやすい,シャープ_個性的である,シャープ_洗練されてる,シャープ_都会的である,シャープ_先進性がある,シャープ_性能が優れている,シャープ_デザインが良い,...,生活全般について_友人たちと何かをするときは，誘うことより誘われることの方が多い,生活全般について_人に助言するのは得意なほうではないと思う,生活全般について_自分は熱しやすく冷めやすいタイプだと思う,生活全般について_多くの人が訪れる話題のスポットなどへ出かけるのが好きだ,生活全般について_刺激的だがリスクがある生活よりも，リスクの少ない安定した生活を送りたいと思う,生活全般について_他人と争ってでも自己主張するより，自己主張せずに周囲とのトラブルを避けたい,生活全般について_余暇時間を大事にし，充実させたいと強く思う,生活全般について_仕事が生きがいで，仕事中心に生活したいと思う,生活全般について_自分が周囲からどのように思われるかが気になる,生活全般について_周囲から一目置かれたいと思う
0,1,NaN,1,2,2,2,1,1,1,2,...,2,1,1,2,2,2,2,1,2,2
1,2,NaN,4,4,4,4,4,4,4,4,...,5,5,4,4,2,2,5,5,5,4
2,3,NaN,4,4,2,3,3,2,3,2,...,4,3,5,2,3,2,4,4,4,4
3,4,NaN,5,5,4,5,3,5,5,5,...,4,2,4,1,4,4,5,2,4,5
4,5,NaN,3,2,1,3,2,2,3,3,...,1,3,2,2,2,3,1,3,2,3


In [9]:
# 男性と女性で各ブランドの平均イメージ評価の違いを可視化する

import matplotlib.pyplot as plt

comp_names = ["シャープ", "ソニー ","パナソニック","日立", "東芝", "三菱電機"]

for comp_name in comp_names:
    comp_cols = [col for col in df.columns if col.startswith(f"{comp_name}_")]
    if not comp_cols:
        continue
    # 男性・女性ごとに平均を計算
    male_means = df[df["性別"] == True][comp_cols].mean()
    female_means = df[df["性別"] == False][comp_cols].mean()
    x = range(len(comp_cols))
    plt.figure(figsize=(10, 6))
    plt.bar(x, male_means, width=0.4, label="男性", align='center', alpha=0.7)
    plt.bar([i + 0.4 for i in x], female_means, width=0.4, label="女性", align='center', alpha=0.7)
    plt.xticks([i + 0.2 for i in x], [col.replace(f"{comp_name}_", "") for col in comp_cols], rotation=45, ha="right")
    plt.title(f"ブランド『{comp_name}』に対する男女別平均イメージ評価")
    plt.ylabel("平均スコア")
    plt.ylim(2.5, 4.5)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"output/comp_eval_means_gender/comp_means_gender_{comp_name}.png")
    #plt.show()
    plt.close()
